#### Import the required libraries we need for the lab.


In [ ]:
import piplite
await piplite.install(['numpy'],['pandas'])
await piplite.install(['seaborn'])

In [ ]:
import pandas as pd
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as pyplot
import scipy.stats
import statsmodels.api as sm
from statsmodels.formula.api import ols

#### Read the dataset in the csv file from the URL


In [ ]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ST0151EN-SkillsNetwork/labs/boston_housing.csv'
resp = await fetch(URL)
boston_url = io.BytesIO((await resp.arrayBuffer()).to_py())

In [ ]:
boston_df=pd.read_csv(boston_url)

#### Add your code below following the instructions given in the course to complete the peer graded assignment


In [ ]:
await piplite.install(['seaborn'])
from scipy import stats

In [ ]:
# Use a boxplot of median values of owner-occupied houses.
bxplt = sns.boxplot(y="MEDV",
                 data=boston_df)
plt.show()

Visualize homes that are/are not bordered by the Charles River.

In [ ]:
# Count the occurrences of each value in the Charles River variable
charles_river_counts = boston_df['CHAS'].value_counts()
charles_river_counts.plot(kind='bar', color=['blue', 'orange'])
plt.xlabel('Borders Charles River (1 = Yes, 0 = No)')
plt.ylabel('Count of Towns')
plt.xticks(ticks=[0, 1], labels=['No', 'Yes'], rotation=0)
plt.show()


### Is there a significant difference in median value of houses bounded by the Charles river?

Conduct levene test for significance in order to use a t-test to test
whether houses bounded by the Charles River have a significanct difference in median values of owner-occupied homes. 


In [ ]:
# Create a histogram to visualize distribution of the 'MEDV' variable - median value of owner-occupied homes.
ax = sns.distplot(boston_df['MEDV'],
                  bins=20,
                  kde=True,
                  color='red',
                  hist_kws={"linewidth": 15,'alpha':1})
ax.set(xlabel='Normal Distribution', ylabel='Frequency')

State the Hypothesis:
*   $H\_0: µ\_1 = µ\_2$ ("there is no difference in median values of houses bounded by the Charles River")
*   $H\_1: µ\_1 ≠ µ\_2$ ("there is a difference in median values of houses bounded by the Charles River")


(α = 0.05)

In [ ]:
bound = boston_df[boston_df['CHAS'] == 1]['MEDV']  # Houses bounded by Charles River
unbound = boston_df[boston_df['CHAS'] == 0] ['MEDV'] #Houses unbounded
stats.levene(bound, unbound)

stats.ttest_ind(bound, unbound, equal_var=(levene_p > 0.05))

**Conclusion:**  The null hypothesis must be rejected as the P-value < 0.05. There is a significant diference in median values of houses bounded by the Charles River. 

### Is there a difference in Median values of houses for each proportion of owner occupied units built prior to 1940?
Discretize units built prior to 1940 into 3 different categories and portray boxplot. 

In [ ]:
boston_df.loc[(boston_df['AGE'] <=35), 'agegroup'] = '35 years and under'
boston_df.loc[(boston_df['AGE'] > 35)&(boston_df['AGE'] < 70), 'agegroup'] = 'between 35 and 70 years'
boston_df.loc[(boston_df['AGE'] >= 70), 'agegroup'] = '70 years and older'

#Box plot of age categories vs. median values 
bxplt = sns.boxplot(x="MEDV", y="agegroup",
                 data=boston_df)
plt.show()

State the Hypothesis:
*   $H\_0: µ\_1 = µ\_2$ ("there is no difference in median values of houses for each proportion of owner occupied units before 1940")
*   $H\_1: µ\_1 ≠ µ\_2$ ("there is a difference in median values of houses for each proportion of owner occupied units before 1940")


(α = 0.05)

In [ ]:
thirtyfive_lower = boston_df[boston_df['agegroup'] == '35 years and under']['MEDV']
btwnthirtyfive_seventy = boston_df[boston_df['agegroup'] == 'between 35 and 70 years']['MEDV']
seventy_up= boston_df[boston_df['agegroup'] == '70 years and older']['MEDV']

scipy.stats.f_oneway(thirtyfive_lower, btwnthirtyfive_seventy, seventy_up)

**Conclusion:** We reject the null hypothesis as the P-value is less than 0.05. There is a difference in median values of houses for each proportion of owner occupied units before 1940.

### Can we conclude that there is no relationship between Nitric Oxide concentrations and proportion of non-retail business acres per town? 
Use a scatter plot to visualize the two continuous variables.

In [ ]:
sns.scatterplot(x="INDUS", y="NOX", data=boston_df)

*   $H\_0:$ There is not a relationship between Nitric Oxide concentrations and propotion of non-retail business acres per town
*   $H\_1:$ There is a relationship between Nitric Oxide concentrations and propotion of non-retail business acres per town

Use the Pearson Correlation method to test the relationship between the two variables:

In [ ]:
scipy.stats.pearsonr(boston_df['INDUS'], boston_df['NOX'])

**Conclusion:** The Pearson coefficient is 0.7636 and greater than α, which means we fail to reject the null hypothesis and there is an association between the two Nitric oxide concentrations and the prpotion of non-retail business acres per town. 

### What is the impact of an additional weighted distance  to the five Boston employment centres on the median value of owner occupied homes? (Regression analysis)



*   $H\_0: β1$ = 0 There is no impact from distance to the five Boston on median values of homes
*   $H\_1: β1$ is not equal to 0 (There is an impact from distance to the employment centres on the median values of homes)

In [ ]:
X = boston_df['DIS']
y = boston_df['MEDV']
X = sm.add_constant(X) 

model = sm.OLS(y, X).fit()
predictions = model.predict(X)
# Print out the statistics
model.summary()

**Conclusion:** The p-value < (α) level = 0.05, so we reject the null hypothesis, as there is evidence that there is an impact on median values of houses according to their distance to employment centers. The coefficient articulates that additional weighted distance to the five Boston employment centers the median value of homes decreased the values by $1,091.6.